In [1]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
output_parser = CommaSeparatedListOutputParser()

In [10]:
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List ten {subject}.\n{format_instructions}",
    input_variables=['subject'],
    partial_variables={'format_instructions' : format_instructions}
)

model = ChatOpenAI(
    model='gpt-4.1',
    temperature=0
)

chain = prompt  |  model  | output_parser

In [11]:
chain.invoke({"subject" : "팔당댐 근처 맛집"})

['팔당초계국수',
 '팔당원조능이버섯칼국수',
 '팔당연가',
 '팔당가든',
 '팔당닭발',
 '팔당숯불장어',
 '팔당민물매운탕',
 '팔당한정식',
 '팔당초밥',
 '팔당막국수']

In [12]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [13]:
response_list = [
    ResponseSchema(name='answer', description="사용자 질문에 대한 답변"),
    ResponseSchema(
        name='source',
        description='사용자의 질문에 답하기 위해 사용된 `출처`, `웹사이트주소` 이어야 한다'
    )
]

In [14]:
output_parser = StructuredOutputParser.from_response_schemas(response_list)

In [17]:
format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="answer the users question as best as possible.\n{format_instructions}\n{question}",
    input_variables=['question'],
    partial_variables={'format_instructions' :format_instructions }
)

In [18]:
chain = prompt  |  model  | output_parser

In [22]:
text = input("질의 : ")
chain.invoke({"question" : text})

{'answer': '아리계곡 주점은 강원도 정선군에 위치한 아리계곡 인근에 있는 음식점 또는 주점으로, 계곡의 자연경관을 감상하며 식사나 음료를 즐길 수 있는 곳입니다. 특히 여름철에는 계곡 물놀이와 함께 방문하는 사람들이 많으며, 주로 산채비빔밥, 막걸리, 닭백숙 등 지역 특색 음식을 제공합니다. 정확한 위치나 영업 정보는 방문 전 확인이 필요합니다.',
 'source': 'https://map.naver.com/p/search/%EC%95%84%EB%A6%AC%EA%B3%84%EA%B3%A1%20%EC%A3%BC%EC%A0%90'}

In [24]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

In [25]:
class Topic(BaseModel):
    description : str = Field(description='주제에 대한 간결한 설명'),
    hashtags : str = Field(description="해시태그 형식의 키워드(2개 이상)")

In [26]:
parser = JsonOutputParser(pydantic_object=Topic)

In [27]:
from langchain_core.prompts import ChatPromptTemplate


In [38]:
prompt = ChatPromptTemplate(
    [
        ("system", "당신은 친구같은 AI 어시스턴트입니다. 당신은 여성입니다."),
        ("user", "#FORMAT: {format_instructions}\n\n#Question: {question}")
    ]
)

In [39]:
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

/home/play/miniconda3/envs/openai/lib/python3.12/site-packages/pydantic/json_schema.py:2324: PydanticJsonSchemaWarning: Default value (FieldInfo(annotation=NoneType, required=True, description='주제에 대한 간결한 설명'),) is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [40]:
chain = prompt | model | parser

In [41]:
text = input("질의 :")
chain.invoke({"question" : text})

{'description': '오늘 점심 메뉴로는 김치찌개, 제육볶음, 샐러드, 또는 비빔밥을 추천해! 한식이 땡기지 않는다면 파스타나 샌드위치도 좋아.',
 'hashtags': '#점메추 #점심추천 #오늘뭐먹지 #한식 #양식'}